In [1]:
# from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2FeatureExtractor
from transformers import AutoModel
import torch
from torch import nn
import numpy as np

from auditory_cortex.dataloader import DataLoader

2024-06-27 23:48:12.460347: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 23:48:14.155512: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/gilbreth/cuda-toolkit/cuda-11.2.0/extras/CUPTI/lib64:/apps/gilbreth/cuda-toolkit/cuda-11.2.0/lib64:/apps/spack/gilbreth/apps/intel-mpi/2017.1.132-intel-17.0.1-p7yx74h/compilers_and_libraries_2017.1.132/linux/mpi/intel64/lib:/apps/spack/gilbreth/apps/intel-mpi/2017.1.132-intel-17.0.1-p7yx74h/compilers_and_libraries_2017.1.132/linux/mpi/mic/lib:/apps/spack/gilbreth/apps/intel-parallel-s

In [2]:
# loading our model weights
model = AutoModel.from_pretrained("m-a-p/MERT-v1-330M", trust_remote_code=True)
# loading the corresponding preprocessor config
processor = Wav2Vec2FeatureExtractor.from_pretrained("m-a-p/MERT-v1-330M",trust_remote_code=True)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /m-a-p/MERT-v1-330M/resolve/main/config.json HTTP/1.1" 200 0
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /m-a-p/MERT-v1-330M/resolve/main/configuration_MERT.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /m-a-p/MERT-v1-330M/resolve/main/config.json HTTP/1.1" 200 0
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /m-a-p/MERT-v1-330M/resolve/main/modeling_MERT.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /m-a-p/MERT-v1-330M/resolve/main/configuration_MERT.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://hugging

In [3]:
dataloader = DataLoader()

Using default normalizer file...


In [4]:
aud = dataloader.get_stim_aud(12)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
from scipy.signal import resample

aud = resample(aud, int(aud.size*24 / 16))

In [7]:
sampling_rate = 24000

In [8]:
model = model.to(device)

In [9]:

import numpy as np
input = aud.astype(np.float64)
input_values = processor(input, sampling_rate=sampling_rate, return_tensors="pt", padding="longest").input_values  # Batch size 1
model.eval()
# with torch.no_grad():
input_values = input_values.to(device)
out = model(input_values)

In [13]:
dict([*model.named_modules()])['encoder.layers']

ModuleList(
  (0-23): 24 x HubertEncoderLayerStableLayerNorm(
    (attention): HubertAttention(
      (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (dropout): Dropout(p=0.0, inplace=False)
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (feed_forward): HubertFeedForward(
      (intermediate_dropout): Dropout(p=0.0, inplace=False)
      (intermediate_dense): Linear(in_features=1024, out_features=4096, bias=True)
      (intermediate_act_fn): GELUActivation()
      (output_dense): Linear(in_features=4096, out_features=1024, bias=True)
      (output_dropout): Dropout(p=0.0, inplace=False)
    )
    (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
)